# Introduction

## Overall concept
The topic that our team settled on is Undergrad course selection in UW Madison. Selecting the appropiate courses for every semester can be a challenging process to some students who aren't certain about the degree structure and numerous requirements. 

Enrollement variables that are considered are:
1. Prerequisites - courses that are required to be completed prior to enrolling in certain classes
2. Semester workload capacity - number of courses taken any given semester cannot exceed a certain number of course credit hours
3. Compulsory courses - courses that are required to be taken at somepoint prior to graduation

Given that students picked their courses for every semester without considering future courses that have strict prerequisites, some of which are compulsory courses, it is very likely that the student will pick a non optimum route for graduation. For example, a student will not be able to enroll in a compulsory course in a later semester if he/she haven't taken the prerequisites to that course. As a result, he/she will have to enroll in the prerequisite courses before being able to enroll in the compulsory courses in subsequent semesters, whcih will delay the student's graduation date. 

The goal of this project is to find an optimum course selection for each semester in regards to completing the graduation requirements within the shortest time period and least number of courses taken. The optimization technique used in this project are linear and logical optimization

** Should we add these feature?**
1. Desired courses - 
Through the interactive console, the student will be able to specify which courses must be included in the plan. 
2. Limit workload - 
Through the interactive console, the student will be able to cap the maximum credit for the specified semester. 

## Data
The requirements for undergraduate CS certificate can be found at https://guide.wisc.edu/undergraduate/letters-science/computer-sciences/computer-sciences-ba/#requirementstext. 

The course list is obtained by web scrapping the course search and enroll website. The list of all relevant courses will be generated in this step. 

One limitation to our program is that the course prerequisites and compulsory courses have to be manually added through code. 

## Outline
1. Mathematical model
2. Data preparation
3. Constructing the Optimization Problem
4. Results and discussion
5. Conclusion

# Mathematical Model

### Notations

$T$ is the max number of maximum number of semesters (e.g., $T = 8$ means the entire 4-year college).

$ t\in \{1, \cdots, T\}$ stands for a specific semester (e.g., $t = 2$ means the second semester or the Spring semester in the first year).

$C$ is the set of classes (e.g., $C = \{ \text{CS 200}, \text{CS 300}, \text{CS 524}, \cdots \}$).

$c \in C$ stands for a specific class (e.g., $c = \text{CS 524}$)

### Decision variables

$x[t, c]$ is a Boolean variable to denote whether to take the class $c \in C$ at semester $t \in \{1, \cdots, T\}$. 

For example, if a student takes CS 524 on the thrid semester, then $x[3, \text{CS 524}]= 1$


### Prerequisite constraint

To take the class $c$, students may need to take $c'$ in the previous semesters, we can encode such prerequisite using the following constraint: 

$$x[t, c] \le \sum_{i=1}^{t - 1} x[i, c'] $$

For example, CS 524 has the prerequisite similar to "(CS 200 or 300) and (MATH 340 or 341)", we can encode it with the constraint:

$$x[t, \text{CS 524}] \le \sum_{i=1}^{t - 1} x[i, \text{CS 200}] + \sum_{i=1}^{t - 1} x[i, \text{CS 300}] $$

$$x[t, \text{CS 524}] \le \sum_{i=1}^{t - 1} x[i, \text{CS 340}] + \sum_{i=1}^{t - 1} x[i, \text{CS 341}] $$


### Graduation requirement

In order to meet the graduation requirement, some class $c$ must be taken, so we have the constraint:

$$\sum_{t=1}^{T} x[t, c] \ge 1$$
    
There are other kinds of graduation requirements, such as taking $k$ coureses from a list of $C$, we can encode such requirements using the following constraint:

$$\sum_{t=1}^{T} \sum_{c \in C} x[t, c] \ge k$$
    
### Max credits

For undergraduate, the number of credits taken in a given semester cannot exceeds a certain number, so we need to ensure that

$$\sum_{c \in C} x[t, c] \cdot \text{credit}(c) \le \text{max_credit}, \quad \text{for all semester } t$$

### No retaking

Generally, we don't want to take the same class multiple times, so we have the following constraint to ensure that a class is taken at most once.

$$\sum_{t=1}^{T} x[t, c] \le 1, \quad \text{for all class } c$$

### Objective 

A naive objective would be to minimize the sum of $x$: 

$$\min \sum_{i=t}^{T} \sum_{c \in C} x[t, c]$$

But this would lead to many equally good solutions. To avoid this, we add a weight $t$ to the class $c$ if its taken at semester $t$

$$\min \sum_{i=t}^{T} \sum_{c \in C} x[t, c] \cdot t$$

This avoid students procrastinating class to later semesters. 

# Data Preparation

We first load the json file obtained from the UW-Madison [Course Search & Enrollment website](enroll.wisc.edu), and extract all the CS courses.

In [5]:
using Pkg
Pkg.add("JSON")

   Updating registry at `~/.julia/registries/General`
######################################################################### 100.0%
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`


In [1]:
using JSON

raw_data = JSON.parsefile("data/1214-spring-2021.json")["hits"]

cls_data = Dict(
    Symbol(c["courseId"]) => 
    c for c in raw_data 
    if c["subject"]["shortDescription"] in ["COMP SCI", "MATH"] && parse(Int, c["catalogSort"]) <= 700
)

cls_to_id_dict = Dict(
    c["courseDesignation"] => Symbol(c["courseId"])
    for c in raw_data
)

function get_cls_name(id)
    cls = cls_data[id]
    subjects = [e["shortDescription"] for e in cls["allCrossListedSubjects"]]
    if length(subjects) == 0
        return cls["courseDesignation"]
    else
        return join(subjects, "/") * " " * cls["catalogNumber"]
    end
end


cs_cls_str = join(sort([get_cls_name(id) for id in keys(cls_data)]), ", ")
println("We have ", length(cls_data), " undergraduate courses in total: ", cs_cls_str)


We have 116 undergraduate courses in total: B M I/COMP SCI 567, B M I/COMP SCI 576, COMP SCI 200, COMP SCI 220, COMP SCI 270, COMP SCI 298, COMP SCI 300, COMP SCI 304, COMP SCI 310, COMP SCI 319, COMP SCI 320, COMP SCI 368, COMP SCI 368, COMP SCI 368, COMP SCI 369, COMP SCI 400, COMP SCI 402, COMP SCI 407, COMP SCI 412, COMP SCI 520, COMP SCI 534, COMP SCI 536, COMP SCI 537, COMP SCI 538, COMP SCI 540, COMP SCI 542, COMP SCI 559, COMP SCI 564, COMP SCI 570, COMP SCI 577, COMP SCI 638, COMP SCI 638, COMP SCI 638, COMP SCI 638, COMP SCI 639, COMP SCI 639, COMP SCI 639, COMP SCI 640, COMP SCI 642, COMP SCI 679, COMP SCI 681, COMP SCI 682, COMP SCI 699, COMP SCI/CURRIC 502, COMP SCI/DS 579, COMP SCI/E C E 252, COMP SCI/E C E 352, COMP SCI/E C E 354, COMP SCI/E C E 506, COMP SCI/E C E 533, COMP SCI/E C E 552, COMP SCI/E C E/I SY E 524, COMP SCI/E C E/M E 532, COMP SCI/E C E/M E 539, COMP SCI/E C E/MATH 435, COMP SCI/I SY E/M E 558, COMP SCI/I SY E/MATH 425, COMP SCI/I SY E/MATH/STAT 525, CO

In [1]:
for c in raw_data 
    if c["subject"]["shortDescription"] in ["COMP SCI", "MATH"] && parse(Int, c["catalogSort"]) <= 700
        cls = cls_data[Symbol(cls_to_id_dict[c["courseDesignation"]])]
        subjects = [e["shortDescription"] for e in cls["allCrossListedSubjects"]]
        println(c["courseDesignation"])
        if length(subjects) == 0
            println(cls["courseDesignation"], "\n\n\n")
        else
            println(join(subjects, "/") * " " * cls["catalogNumber"], "\n\n\n")
        end
    end
    
end

LoadError: UndefVarError: raw_data not defined

In [2]:
get_math_cls_id(number) =
    get(cls_to_id_dict, "MATH " * string(number), Symbol())

get_cs_cls_id(number) =
    get(cls_to_id_dict, "COMP SCI " * string(number), Symbol())

cs524_data = cls_data[get_cs_cls_id(524)]
join(keys(cs524_data), ", ")

"honors, allCrossListedSubjects, breadths, matched_queries, termCode, levels, subjectAggregate, courseId, academicGroupCode, gradingBasis, advisoryPrerequisites, ethnicStudies, lettersAndScienceCredits, approvedForTopics, courseDesignationRaw, openToFirstYear, gradCourseWork, catalogPrintFlag, sustainability, instructorProvidedContent, courseRequirements, subject, repeatable, fullCourseDesignationRaw, typicallyOffered, foreignLanguage, enrollmentPrerequisites, titleSuggest, minimumCredits, title, lastUpdated, workplaceExperience, courseDesignation, generalEd, topics, description, lastTaught, creditRange, catalogSort, currentlyTaught, firstTaught, catalogNumber, maximumCredits, fullCourseDesignation"

# Constructing the Optimization Problem

### Take in user input as parameter and use it in the following optimization model

In [10]:

sem2course_constraint = []
trnfedCourses = []


function responseOne(sem2course_constraint)
    print("format:\n eg: \"2, COMP SCI 300\"\n")
    response = readline()
    if(size(split(response, ','))[1] != 2)
        print("Invalid format")
        return sem2course_constraint
    end
    
    
    semester = split(response, ',')[1]
    courseName = split(response, ',')[2]
    
    re = r"^([0-9])";
    if(!occursin(re, semester))
        println("Invalid semester number format: \n")
        return sem2course_constraint
    end
    
    # remove space at the beginning if the user input space
    if(courseName[1] == ' ')
        courseName = courseName[2:sizeof(courseName)]
    end
    
    if(!haskey(cls_to_id_dict, courseName))
        println("Invalid course name/ Course not found")
        return sem2course_constraint
    end
    
    println(courseName, "is successfully limited to semester ", semester, "\n")
    return vcat(sem2course_constraint, (parse(Int64, semester), courseName))
end


function responseTwo(trnfedCourses)
    print("format:\n eg: \"COMP SCI 300\"\n")
    
    courseName = readline()
    
    # remove space at the beginning if the user input space
    if(courseName[1] == ' ')
        courseName = courseName[2:sizeof(courseName)]
    end
    
    if(!haskey(cls_to_id_dict, courseName))
        println("Invalid course name/ Course not found")
        return trnfedCourses
    end
    
    println(courseName, " is successfully added to transfered courses list\n ")
    return vcat(trnfedCourses, (courseName))
end

loopV = true

while(loopV)
    
    print("What would you like to do? (enter number)\n
        1. Add X course you wish to take in semester Y 
        2. Add prior transferred courses 
        3. Run program
        \n") 
  
    # Calling rdeadline() function
    response = readline()
    
    if (response == "1")
        sem2course_constraint = responseOne(sem2course_constraint)
        
    elseif(response == "2")
        trnfedCourses = responseTwo(trnfedCourses)
        
    elseif(response == "3")
        loopV = false
    else
        print("invalid response \n")
    end
    
end


What would you like to do? (enter number)

        1. Add X course you wish to take in semester Y 
        2. Add prior transferred courses 
        3. Run program
        
stdin> 3


In [7]:
sem2course_constraint = []

function responseOne(sem2course_constraint)
    print("format:\n eg: \"2, COMP SCI 300\"\n")
    response = readline()
    if(size(split(response, ','))[1] != 2)
        print("Invalid format")
        return sem2course_constraint
    end
    
    
    semester = split(response, ',')[1]
    courseName = split(response, ',')[2]
    
    re = r"^([0-9])";
    if(!occursin(re, semester))
        println("Invalid semester number format: \n")
        return sem2course_constraint
    end
    
    # remove space at the beginning if the user input space
    if(courseName[1] == ' ')
        courseName = courseName[2:sizeof(courseName)]
    end
    
    if(!haskey(cls_to_id_dict, courseName))
        println("Invalid course name")
        return sem2course_constraint
    end
    
    return vcat(sem2course_constraint, (parse(Int64, semester), courseName))
end

responseOne(sem2course_constraint)



format:
 eg: "2, COMP SCI 300"
stdin> 2, COMP SCI 300


1-element Array{Any,1}:
 (2, "COMP SCI 300")

In [7]:
using JuMP, Gurobi

C = keys(cls_data)

"""
Convert a list of CS course numbers to a list of course ids
"""
cs_ids(arr...) =
    filter(id -> id in C, [get_cs_cls_id(e) for e in arr])
"""
Convert a list of Math course numbers to a list of course ids
"""
math_ids(arr...) =
    filter(id -> id in C, [get_math_cls_id(e) for e in arr])

"""
Returns the minimum number of credit given a course id
"""
credit(c) = cls_data[c]["minimumCredits"]

"""
Returns the full name for a course given the course id
"""
get_cls_full_name(id) = 
    get_cls_name(id) * ": " * cls_data[id]["title"]

function _add_prereq(id1, id2, cls, prereq)
    for t in T
        prereq_id = filter(id -> id in C, [id2(p) for p in prereq])
        @constraint(m, x[id1(cls), t] <= sum(x[id, i] for i in 1:t-1 for id in prereq_id))
    end
end

"""
Specify CS class prerequisite for a given CS class
"""
add_cs_cs_prereq(cs_cls, one_of...) = 
    _add_prereq(get_cs_cls_id, get_cs_cls_id, cs_cls, one_of)


"""
Specify Math class prerequisite for a given CS class
"""
add_cs_math_prereq(cs_cls, one_of...) =
    _add_prereq(get_cs_cls_id, get_math_cls_id, cs_cls, one_of)

"""
new!! check please
Specify Math class prerequisite for a given Math class
"""
add_math_math_prereq(math_cls, one_of...) =
    _add_prereq(get_math_cls_id, get_math_cls_id, math_cls, one_of)


"""
Add all course prerequisite
"""
function add_all_prereq()
    add_cs_cs_prereq(300, 200)
    add_cs_cs_prereq(354, 252)
    add_cs_cs_prereq(354, 300)

    add_cs_cs_prereq(506, 400)
    add_cs_cs_prereq(506, 407, 536, 537, 559, 564, 570, 679, 552)

    add_cs_cs_prereq(552, 352)
    add_cs_cs_prereq(552, 354)
    
    add_cs_cs_prereq(559, 400)

    for c in [400, 407, 513, 514, 524, 534, 540, 570]
        add_cs_cs_prereq(c, 300)
    end

    for c in [536, 537, 538, 564]
        add_cs_cs_prereq(c, 354)
        add_cs_cs_prereq(c, 400)
    end

    for c in [520, 577]
        add_cs_cs_prereq(c, 400)
        add_cs_cs_prereq(c, 240, 475)
    end

    for c in [640, 642]
        add_cs_cs_prereq(c, 537)
    end
    
    for c in [513, 524, 525]
        add_cs_math_prereq(c, 340, 341, 375)
    end
    
    #new!!!! check please
    add_math_math_prereq(322,321)
    #new
end

"""
See https://guide.wisc.edu/undergraduate/letters-science/computer-sciences/computer-sciences-ba/#requirementstext
"""
function add_cs_graduation_req()
    C_basic = cs_ids(240, 252, 300, 354, 400)
    C_theory = cs_ids(577, 520)
    C_xware = cs_ids(407, 506, 536, 538, 537, 552, 564, 640, 642)
    C_app = cs_ids(412, 425, 513, 514, 524, 525, 534, 540, 545, 547, 559, 570)
    C_elec = cs_ids(
        407, 412, 425, 435, 471, 475, 506, 513, 514, 520, 524, 525, 526, 532, 
        533, 534, 536, 537, 538, 539, 540, 545, 547, 552, 558, 559, 564, 567, 
        570, 576, 577, 579, 635, 640, 642, 679, 639)

    
    # Take all from basic computer sciences
    for c in C_basic
        @constraint(m, sum(x[c, t] for t in T) >= 1)
    end

    # Complete 1 for Theory of computer science
    @constraint(m, sum(x[c, t] for t in T for c in C_theory) >= 1)

    # Complete 2 for Software & Hardware
    @constraint(m, sum(x[c, t] for t in T for c in C_xware) >= 2)

    # Complete 1 for Applications
    @constraint(m, sum(x[c, t] for t in T for c in C_app) >= 1)

    # Complete 2 for Electives
    @constraint(m, sum(x[c, t] for t in T for c in C_elec) >= 2)
end


function add_math_graduation_req()#at least 30 credit hours
    #math core course for at least 18 credits
    M_linear_algebra = math_ids(320,341,340) #no math 375 in data ignore this requirement
    M_intermediate_mathematics = math_ids(421,467)
    M_intermediate_mathematics_A = math_ids(321,322) # a combine choice can be instead of 1 course in M_intermediate_mathematics
    M_advanced_mathematics = math_ids(514,521,531,535,540,541,571)
    M_math_elective_req_A = math_ids(
        513,522,525,542,567,570,
        605,619,627,629,632,635)
    M_math_elective_req_B = math_ids(
        310,319,376,415,425,431,309,435,
        443,475)
    
    #Programming and Computations Requirement at least 12 credit hours
    C_1 = cs_ids(300)
    C_2 = cs_ids(400)
    C_elective = cs_ids(
        412,471,520,
        524,526,532,533,534,538,539,
        540,545,558,559,567,576,577,635,
        642)


    #Constraints
    
    #math core course for at least 18 credits
    # Complete 1 for linear_algebra
    @constraint(m, sum(x[c, t] for t in T for c in M_linear_algebra) >= 1)
    # Complete 1 or 2 for intermediate_mathematics or intermediate_mathematics _A
    for t in T
        if x[Symbol("011647"), t] == 1      #Symbol("011647") is math 321 
            @constraint(m, sum(x[c, t] for t in T for c in M_intermediate_mathematics_A) >=2)
        else
            @constraint(m, sum(x[c, t] for t in T for c in M_intermediate_mathematics) >=1)
        end
    end
    
    @constraint(m, sum(x[c, t] for t in T for c in M_intermediate_mathematics) >= 1)
    # Complete 1 for advanced_mathematics
    @constraint(m, sum(x[c, t] for t in T for c in M_advanced_mathematics) >= 1)
    # Complete 1 for math_elective_reqirement_A
    @constraint(m, sum(x[c, t] for t in T for c in M_math_elective_req_A) >= 1)

    
    ########################################################################################
    #no math 375 in data ignore this requirement                                           #
    # if take 375 and take 321 then need 1 course in math_elective_reqirement_B            #
    # if take 375 and not take 321 then need 2 course in math_elective_reqirement_B        #
    # if not take 375 and take 321 then need 1 course in math_elective_reqirement_B        #
    # if not take 375 and not take 321 then need 2 course in math_elective_reqirement_B    #
    ########################################################################################
    #for t in T
    #    if x[math_ids(375), t] == 1
    #        if x[math_ids(321), t] == 1
    #            # Complete 1 for math_elective_reqirement_B
    #            @constraint(m, sum(x[c, t] for t in T for c in M_math_elective_req_B) >= 1)
    #        else
    #            # Complete 2 for math_elective_reqirement_B
    #            @constraint(m, sum(x[c, t] for t in T for c in M_math_elective_req_B) >= 2)
    #        end
    #    else
    #        if x[math_ids(321), t] == 1
    #            # Complete 1 for math_elective_reqirement_B
    #            @constraint(m, sum(x[c, t] for t in T for c in M_math_elective_req_B) >= 1)
    #        else
    #            # Complete 2 for math_elective_reqirement_B
    #            @constraint(m, sum(x[c, t] for t in T for c in M_math_elective_req_B) >= 2)
    #        end
    #end
    #end
    
    
    # Complete 1 or 2 for math_elective_reqirement_B depends on other selections
    for t in T
        if x[Symbol("011647"), t] == 1 && x[Symbol("011648"), t] == 1     #Symbol("011647") is math 321   Symbol("011648") is math 322
            # Complete 1 for math_elective_reqirement_B
            @constraint(m, sum(x[c, t] for t in T for c in M_math_elective_req_B) >= 1)
        else
            # Complete 2 for math_elective_reqirement_B
            @constraint(m, sum(x[c, t] for t in T for c in M_math_elective_req_B) >= 2)
        end
    end
    
    #Programming and Computations Requirement at least 12 credit hours
    # Complete CS_300
    @constraint(m, sum(x[c, t] for t in T for c in C_1) >= 1)
    # Complete CS_400
    @constraint(m, sum(x[c, t] for t in T for c in C_2) >= 1)
    # Complete Programming and Computations 2 elective courses
    @constraint(m, sum(x[c, t] for t in T for c in C_elective) >= 2)
end

function add_common_constraints()
    # Max credit constraint
    for t in T
        @constraint(m, sum(x[c, t] * credit(c) for c in C) <= max_credit)
    end

    # No retaking
    for c in C
        @constraint(m, sum(x[c, t] for t in T) <= 1)
    end
end

function add_objective()
    @objective(m, Min, sum(x[c, t] * t for c in C, t in T))
end

function print_schedule(x)
    for t in T
        for c in C
            if x[c, t] > 0
                println("Semester ", t, ": take \"", get_cls_full_name(c), "\"")
            end
        end
    end
end
;

  Resolving package versions...
  Installed Cgl_jll ─ v0.60.2+6
  Installed Cbc_jll ─ v2.10.5+2
  Installed Cbc ───── v0.8.0
Updating `~/.julia/environments/v1.5/Project.toml`
  [9961bab8] + Cbc v0.8.0
Updating `~/.julia/environments/v1.5/Manifest.toml`
  [9961bab8] + Cbc v0.8.0
  [38041ee0] + Cbc_jll v2.10.5+2
  [3830e938] + Cgl_jll v0.60.2+6
   Building Cbc → `~/.julia/packages/Cbc/JdG60/deps/build.log`
┌ Info: Precompiling Cbc [9961bab8-2fa3-5c5a-9d89-47fab24efd76]
└ @ Base loading.jl:1278


OPTIMAL
Semester 1: take "COMP SCI 200: Programming I"
Semester 1: take "COMP SCI 252: Introduction to Computer Engineering"
Semester 2: take "COMP SCI 240: Introduction to Discrete Mathematics"
Semester 2: take "COMP SCI 300: Programming II"
Semester 3: take "COMP SCI 534: Computational Photography"
Semester 3: take "COMP SCI 407: Foundations of Mobile Systems and Applications"
Semester 4: take "COMP SCI 400: Programming III"
Semester 4: take "COMP SCI 354: Machine Organization and Programming"
Semester 5: take "COMP SCI 506: Software Engineering"
Semester 5: take "COMP SCI 520: Introduction to Theory of Computing"
